# LLaMA on SageMaker JumpStart — Clean Room Tutorial (My Reference Copy)
> **Purpose of this notebook:** a **self-contained, reproducible** guide to deploy a LLaMA model on **Amazon SageMaker JumpStart**.  
> **Why this exists:** to avoid confusing it with my older Falcon notebook; this one is **LLaMA-only** and uses **JumpStart helpers**.
---
## What you’ll do (TL;DR)
1. Pick a **LLaMA** JumpStart model ID valid in **my AWS region**.
2. Use **JumpStart helper utilities** to fetch image+artifacts.
3. Deploy an **inference endpoint**.
4. Run **test prompts** and adjust parameters.
5. **Monitor & clean up**.
> **Note**: LLaMA models require acceptance of Meta’s license/terms. Ensure your account has access before deployment.

## Requirements & Assumptions
- I’m running this in **SageMaker Studio** with an attached execution role that has access to SageMaker, S3, and CloudWatch.
- The **sagemaker** Python SDK is recent (upgrade cell below if needed).
- I have **accepted** the relevant model EULAs/terms (Meta) where required.
- I understand JumpStart model availability can vary by **region**.

In [ ]:
# OPTIONAL: Upgrade the SageMaker SDK if I hit import/JumpStart issues.
# !pip install --upgrade --quiet sagemaker boto3 botocore

## Step 0 — Set AWS region and IAM role
- Region drives which JumpStart **model IDs** are available.
- Role must have permissions to create and invoke endpoints.

In [ ]:
import os, boto3, sagemaker
from sagemaker import Session
session = Session()
region = session.boto_region_name or os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
try:
    from sagemaker import get_execution_role
    role = get_execution_role()
except Exception:
    role = os.environ.get("SAGEMAKER_EXECUTION_ROLE_ARN", "arn:aws:iam::<ACCOUNT_ID>:role/<SageMakerExecutionRole>")
print(f"Region: {region}\nRole:   {role}")

## Step 1 — Pick the **LLaMA** JumpStart model ID (region-specific)
Verify in SageMaker Studio JumpStart UI before running.
- Example IDs:
  - `meta-textgeneration-llama-3-8b-instruct`
  - `meta-textgeneration-llama-2-7b-instruct`

In [ ]:
model_id = "meta-textgeneration-llama-3-8b-instruct"  # <- change if needed
model_version = "*"

## Step 2 — Retrieve JumpStart assets (image URI, model artifact, environment)

In [ ]:
from sagemaker import image_uris, model_uris, environment_variables
inference_image_uri = image_uris.retrieve(model_id=model_id, model_version=model_version, image_scope="inference", region=region)
model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, region=region)
env = environment_variables.retrieve(model_id=model_id, model_version=model_version, region=region)
print("Image URI:", inference_image_uri)
print("Model URI:", model_uri)

## Step 3 — Create the SageMaker Model & Deploy the Endpoint

In [ ]:
from sagemaker.model import Model
sm_model = Model(image_uri=inference_image_uri, model_data=model_uri, role=role, env=env, sagemaker_session=session)
endpoint_name = f"llama-jumpstart-endpoint"
instance_type = "ml.g5.xlarge"
predictor = sm_model.deploy(endpoint_name=endpoint_name, initial_instance_count=1, instance_type=instance_type)

## Step 4 — Invoke the Endpoint (test prompts)

In [ ]:
payload = {"inputs": "Explain vector databases to a junior developer.", "parameters": {"max_new_tokens": 128, "temperature": 0.7}}
response = predictor.predict(payload)
print(response)

## Step 5 — Clean Up

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)